# Параллельные вычисления

Материалы:
* Макрушин С.В. Лекция 10: Параллельные вычисления
* https://docs.python.org/3/library/multiprocessing.html

## Задачи для совместного разбора

1. Посчитайте, сколько раз встречается каждый из символов (заглавные и строчные символы не различаются) в файле `Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt` и в файле `Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt`. 

In [18]:
dictionary1 = {}
dictionary2 = {}
with open('10_multiprocessing_data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt', 'r') as file:
    text1 = file.read().replace('\n', '').replace('\xa0', ' ')
with open('10_multiprocessing_data/Dostoevskiy Fedor. Prestuplenie i nakazanie - BooksCafe.Net.txt', 'r') as file:
    text2 = file.read().replace('\n', '').replace('\xa0', ' ')


def counter (text):
    dictionary = {}
    for i in text:
        if i.lower() not in dictionary:
            dictionary[i.lower()] = 0
        else:
            dictionary[i.lower()] = dictionary[i.lower()] + 1 
    print(dictionary)

print("Текст 1")
counter(text1)
print("Текст 2")
counter(text2)


Текст 1
{'с': 11506, 'п': 5488, 'а': 18235, 'и': 13586, 'б': 3979, 'о': 23129, ',': 6371, ' ': 46547, 'ч': 4112, 'т': 14244, 'к': 6743, 'л': 9960, 'н': 14239, 'г': 3947, 'у': 6043, 'в': 9397, 'е': 20053, 'й': 2027, 'э': 835, 'р': 9481, 'b': 219, 'o': 376, 'k': 20, 's': 428, 'c': 323, 'a': 589, 'f': 51, 'e': 1199, '.': 2953, 'n': 458, 't': 331, ':': 211, 'h': 226, 'p': 99, '/': 19, 'u': 284, 'r': 307, 'd': 191, 'v': 86, 'i': 368, 'y': 7, '_': 3, '-': 899, '1': 45, '0': 21, '9': 35, '6': 41, 'm': 400, 'l': 570, 'ж': 2296, 'д': 6680, 'х': 1534, 'ф': 633, 'м': 7105, 'g': 72, '2': 41, '4': 41, '7': 41, 'я': 5457, '!': 717, '(': 275, 'з': 3354, ')': 275, 'ц': 816, 'ь': 4856, 'ш': 1942, 'ы': 3868, 'ю': 1322, ';': 405, 'щ': 586, '?': 570, 'ъ': 62, '—': 1725, '…': 279, '«': 128, '[': 96, ']': 96, '»': 127, "'": 58, '3': 39, '5': 39, 'q': 49, 'j': 39, '8': 37, 'w': 6, 'x': 23, 'z': 43, '`': 8}
Текст 2
{'с': 50083, 'п': 25651, 'а': 73554, 'и': 62029, 'б': 16015, 'о': 106739, ',': 26972, ' ': 1883

2. Решить задачу 1, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создать свой собственный процесс. 

In [ ]:
import multiprocessing as mp

def counter(filename, result_dict):
    with open(filename, 'r') as file:
        text = file.read().replace('\n', '').replace('\xa0', ' ')
    dictionary = {}
    for i in text:
        if i.lower() not in dictionary:
            dictionary[i.lower()] = 1
        else:
            dictionary[i.lower()] = dictionary[i.lower()] + 1  
    result_dict.put(dictionary)

output = mp.Queue()
dictionary1 = {}
dictionary2 = {}
processes = [mp.Process(target=counter, args=('10_multiprocessing_data/Dostoevskiy Fedor. Igrok - BooksCafe.Net.txt', output))]

for p in processes:
    p.start()

for p in processes:
    p.join()

results = [output.get() for p in processes]

print(results)
    

## Лабораторная работа 10

1. Разбейте файл `recipes_full.csv` на несколько (например, 8) примерно одинаковых по объему файлов c названиями `id_tag_nsteps_*.csv`. Каждый файл содержит 3 столбца: `id`, `tag` и `n_steps`, разделенных символом `;`. Для разбора строк используйте `csv.reader`.

__Важно__: вы не можете загружать в память весь файл сразу. Посмотреть на первые несколько строк файла вы можете, написав код, который считывает эти строки.

Подсказка: примерное кол-во строк в файле - 2.3 млн.

```
id;tag;n_steps
137739;60-minutes-or-less;11
137739;time-to-make;11
137739;course;11
```


In [28]:
import pandas as pd
skip = 1
rows = 290000
header = {1:'id',5:'tags',6:'n_steps'} 
for i in range (0, 8):
    data = pd.read_csv('recipes_full.csv', header = None, skiprows = skip, nrows=rows)
    skip = skip + rows
    data.iloc[:, [1, 5, 6]].to_csv('id_tag_nsteps_' + str(i) + '.csv', index=False, header=[header[1], header[5], header[6]], decimal=';')


2. Напишите функцию, которая принимает на вход название файла, созданного в результате решения задачи 1, считает среднее значение количества шагов для каждого тэга и возвращает результат в виде словаря.

3. Напишите функцию, которая считает среднее значение количества шагов для каждого тэга по всем файлам, полученным в задаче 1, и возвращает результат в виде словаря. Не используйте параллельных вычислений. При реализации выделите функцию, которая объединяет результаты обработки отдельных файлов. Модифицируйте код из задачи 2 таким образом, чтобы иметь возможность получить результат, имея результаты обработки отдельных файлов. Определите, за какое время задача решается для всех файлов.


4. Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Для обработки каждого файла создайте свой собственный процесс. Определите, за какое время задача решается для всех файлов.

5. (*) Решите задачу 3, распараллелив вычисления с помощью модуля `multiprocessing`. Создайте фиксированное количество процессов (равное половине количества ядер на компьютере). При помощи очереди передайте названия файлов для обработки процессам и при помощи другой очереди заберите от них ответы. 